In [1]:
import pandas as pd
import time
from random import random
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from undetected_chromedriver import Chrome, ChromeOptions
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains

# Scraping en Google Maps

In [2]:
datar = pd.read_csv("Summum_list.txt", sep=",")

In [3]:
listr = datar["Restaurante"].tolist()

In [4]:
listr

['Baan Perú']

In [5]:
options = ChromeOptions()
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")

In [6]:
driver = Chrome(options=options)

In [7]:
driver.get('https://www.google.com/maps')

In [8]:
# Listas para almacenar la información
reviews = []

for restaurant in listr:
    # Ingresar el nombre del restaurante en el campo de búsqueda
    search_bar = driver.find_element(By.ID, "searchboxinput")
    search_bar.clear()
    search_bar.send_keys(restaurant)

    # Presionar el botón de búsqueda
    search_button1 = driver.find_element(By.ID, "searchbox-searchbutton")
    time.sleep(random())
    search_button1.click()

    # Esperar hasta que la página del restaurante cargue
    WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.CLASS_NAME, "a5H0ec")))

    # Hacer clic en el botón "Más opiniones"
    search_button2 = driver.find_elements(By.CLASS_NAME, "HHrUdb")
    for button in search_button2:
        text = button.text
        if "opiniones" in text:
            button.click()
            break
    
    # Esperar hasta que la página de opiniones cargue
    WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.XPATH, '//button[contains(@class, "hh2c6 G7m0Af")][contains(@aria-label, "Revisiones para")]')))
    
    
    # Recopilar la información de las tarjetas de comentarios
    while True:
        cards = driver.find_elements(By.CLASS_NAME, "jftiEf")
        for card in cards:
            name = card.find_element(By.CLASS_NAME, "d4r55").text
            date = card.find_element(By.CLASS_NAME, "rsqaWe").text
            stars = card.find_elements(By.CLASS_NAME, "hCCjke.vzX5Ic")
            num_stars = len(stars)
            
            try:
                more_button = card.find_element(By.CLASS_NAME, "w8nwRe")
                more_button.click()
                time.sleep(1)
                review = card.find_element(By.CLASS_NAME, "wiI7pd").text
            except NoSuchElementException:
                review = card.find_element(By.CLASS_NAME, "wiI7pd").text

            
            # Agregar la información a las listas correspondientes
            reviews.append({
                "Restaurante": restaurant,
                "Usuario": name,
                "Fecha": date,
                "Opinión": review,
                "Valoración": num_stars
            })
            
        # Hacer scroll hacia abajo para cargar más comentarios
        #driver.execute_script("window.scrollBy(0, window.innerHeight);")
        #time.sleep(5)
        
        # Verificar si se han cargado nuevas tarjetas
        new_cards = driver.find_elements(By.CLASS_NAME, "jftiEf")
        if len(new_cards) == len(cards):
            break  # No hay nuevas tarjetas, se ha llegado al final de los comentarios

driver.quit()

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":".wiI7pd"}
  (Session info: chrome=114.0.5735.199)
Stacktrace:
Backtrace:
	GetHandleVerifier [0x00B68893+48451]
	(No symbol) [0x00AFB8A1]
	(No symbol) [0x00A05058]
	(No symbol) [0x00A30467]
	(No symbol) [0x00A3069B]
	(No symbol) [0x00A29631]
	(No symbol) [0x00A4A304]
	(No symbol) [0x00A29586]
	(No symbol) [0x00A4A614]
	(No symbol) [0x00A5C482]
	(No symbol) [0x00A4A0B6]
	(No symbol) [0x00A27E08]
	(No symbol) [0x00A28F2D]
	GetHandleVerifier [0x00DC8E3A+2540266]
	GetHandleVerifier [0x00E08959+2801161]
	GetHandleVerifier [0x00E0295C+2776588]
	GetHandleVerifier [0x00BF2280+612144]
	(No symbol) [0x00B04F6C]
	(No symbol) [0x00B011D8]
	(No symbol) [0x00B012BB]
	(No symbol) [0x00AF4857]
	BaseThreadInitThunk [0x76387D59+25]
	RtlInitializeExceptionChain [0x774EB74B+107]
	RtlClearBits [0x774EB6CF+191]


In [9]:
reviews

[{'Restaurante': 'Baan Perú',
  'Usuario': 'Ale Cáceres',
  'Fecha': 'Hace 4 meses',
  'Opinión': 'El restaurante tiene un ambiente súper, todo en madera y metal y plantas por doquier!\nLa comida estuvo bien hecha, solo me decepcioné un poco por lo estándar que fue la carta, pensé que habrían más cosas de autor.\nRescato el mocktail Sunset, super refrescante. Las conchas a la parrilla con salsa miso fueron espectaculares, regresaría por ellas. Los dumplings estuvieron bien (cangrejo&pimienta y de langostinos), pero nada fuera de lo extraordinario.\nDe postre probamos la galleta, pero tenía tanto chocolate que no sentí para nada el caramelo o el miso.\nLa flor de loto: la crema de cúrcuma con el helado de coco fueron lo mejor! La flor estaba hecha muy gruesa y podía sentir el aceite.\nCreo que este restaurante está súper si son de las primeras excursiones en la comida asiática :)',
  'Valoración': 4},
 {'Restaurante': 'Baan Perú',
  'Usuario': 'Odalis Perea',
  'Fecha': 'Hace 2 meses',


In [10]:
data_rest = pd.DataFrame(reviews)

In [11]:
data_rest['Restaurante'].value_counts()

Baan Perú    114
Name: Restaurante, dtype: int64

In [12]:
data_rest

,Restaurante,Usuario,Fecha,Opinión,Valoración
0,Baan Perú,Ale Cáceres,Hace 4 meses,"El restaurante tiene un ambiente súper, todo e...",4
1,Baan Perú,Odalis Perea,Hace 2 meses,Fui porque vi videos con otros platos que quis...,4
2,Baan Perú,veronica prada,Hace 2 meses,"Un lugar super agradable, la comida fue agrada...",4
3,Baan Perú,Gyan Penrose-Kafka,Hace 5 meses,La comida era sabrosa pero las porciones eran ...,3
4,Baan Perú,Diego Aguirre Vallenas,Hace 5 meses,Lugar muy bien pensado desde el concepto aunqu...,5
...,...,...,...,...,...
109,Baan Perú,Claudia Jo,Hace un año,"Las bebidas con lychee, las mejores!",5
110,Baan Perú,montserrat cabrejos Inchaustegui,Hace un año,Delicioso!,5
111,Baan Perú,Laura Christina,Hace un año,Riquísimo! 😋…,5
112,Baan Perú,Lucas Maldonado,Hace un año,Buena atencion,5


In [13]:
# Descargar la información recopilada

data_rest.to_excel('Informe9_03-07.xlsx', index=True)